In [1]:
import pandas as pd

# ---------- 1. LOAD DATA ----------
warehouses = pd.read_csv("warehouses.csv")
factories = pd.read_csv("factories.csv")
skus = pd.read_csv("skus.csv")
initial_stock = pd.read_csv("initial_stock.csv")
incoming = pd.read_csv("incoming_goods.csv")
movements = pd.read_csv("stock_movements.csv")
distance = pd.read_csv("distance_matrix.csv")

warehouses.head(), incoming.head()


(  warehouse_id         name  capacity_kg   latitude  longitude  \
 0         WH01  Warehouse_1        10000  23.383071  72.756995   
 1         WH02  Warehouse_2         5000  22.747296  72.217867   
 2         WH03  Warehouse_3        15000  23.188958  72.187868   
 3         WH04  Warehouse_4        20000  22.792371  72.316860   
 4         WH05  Warehouse_5        10000  23.042305  72.516956   
 
          sku_types  
 0  oilseeds,spices  
 1              all  
 2              all  
 3  oilseeds,grains  
 4  oilseeds,grains  ,
   incoming_id            timestamp     farmer sku_id  sku_type  quantity_kg  \
 0      IN0001  2025-11-02 09:53:52  Farmer_12  SKU05    spices          100   
 1      IN0002  2025-10-30 10:53:52  Farmer_19  SKU04    spices          100   
 2      IN0003  2025-11-03 13:53:52  Farmer_22  SKU02  oilseeds          300   
 3      IN0004  2025-11-15 10:53:52  Farmer_32  SKU01  oilseeds           50   
 4      IN0005  2025-11-01 13:53:52  Farmer_16  SKU02  oilseeds

In [2]:
# ---------- 2. COMPUTE CURRENT STOCK PER WAREHOUSE ----------

# Start from initial stock
stock = initial_stock.copy()

# Normalize movement types
movements["movement_type"] = movements["movement_type"].str.upper()

# Define positive or negative movement
factor = {
    "INBOUND": 1,
    "TRANSFER_IN": 1,
    "OUTBOUND": -1,
    "TRANSFER_OUT": -1
}

movements["sign"] = movements["movement_type"].map(factor).fillna(0)
movements["effective_qty_kg"] = movements["sign"] * movements["quantity_kg"]

# Sum movement quantities per warehouse + sku
mv_agg = (
    movements.groupby(["warehouse_id", "sku_id"], as_index=False)["effective_qty_kg"]
    .sum()
)

# Merge with initial stock
stock = stock.merge(mv_agg, on=["warehouse_id", "sku_id"], how="left")

stock["effective_qty_kg"] = stock["effective_qty_kg"].fillna(0)
stock["current_quantity_kg"] = stock["quantity_kg"] + stock["effective_qty_kg"]

# Avoid negative quantities
stock.loc[stock["current_quantity_kg"] < 0, "current_quantity_kg"] = 0

stock.head()


,warehouse_id,sku_id,quantity_kg,effective_qty_kg,current_quantity_kg
0,WH01,SKU01,1865,50,1915
1,WH01,SKU02,1133,400,1533
2,WH01,SKU03,807,100,907
3,WH01,SKU04,618,-20,598
4,WH01,SKU05,9,200,209


In [3]:
# ---------- 3. CAPACITY CALCULATION ----------

current_used = (
    stock.groupby("warehouse_id")["current_quantity_kg"]
    .sum()
    .reset_index()
    .rename(columns={"current_quantity_kg": "used_capacity_kg"})
)

warehouses_cap = warehouses.merge(current_used, on="warehouse_id", how="left")
warehouses_cap["used_capacity_kg"] = warehouses_cap["used_capacity_kg"].fillna(0)
warehouses_cap["free_capacity_kg"] = (
    warehouses_cap["capacity_kg"] - warehouses_cap["used_capacity_kg"]
)

warehouses_cap.head()


,warehouse_id,name,capacity_kg,latitude,longitude,sku_types,used_capacity_kg,free_capacity_kg
0,WH01,Warehouse_1,10000,23.383071,72.756995,"oilseeds,spices",6073,3927
1,WH02,Warehouse_2,5000,22.747296,72.217867,all,2649,2351
2,WH03,Warehouse_3,15000,23.188958,72.187868,all,9702,5298
3,WH04,Warehouse_4,20000,22.792371,72.316860,"oilseeds,grains",12787,7213
4,WH05,Warehouse_5,10000,23.042305,72.516956,"oilseeds,grains",8423,1577


In [4]:
# ---------- 4. HELPER DICTIONARIES ----------
capacity_map = warehouses_cap.set_index("warehouse_id")["free_capacity_kg"].to_dict()
sku_types_map = warehouses_cap.set_index("warehouse_id")["sku_types"].to_dict()


In [5]:
# ---------- 5. RECOMMENDATION LOGIC ----------
def recommend_warehouse(row):
    sku_id = row["sku_id"]
    qty = row["quantity_kg"]
    sku_type = row["sku_type"]
    preferred_factory = row["preferred_factory_id"]

    # Candidate warehouses: enough free capacity + SKU compatible
    candidates = []
    for wh_id, free_cap in capacity_map.items():
        if free_cap < qty:
            continue

        wh_sku_types = sku_types_map.get(wh_id, "")
        sku_ok = (sku_type in wh_sku_types) or ("all" in wh_sku_types)

        if not sku_ok:
            continue

        candidates.append(wh_id)

    # If no candidate passes SKU rule → relax rule
    if not candidates:
        candidates = [wh_id for wh_id, free_cap in capacity_map.items() if free_cap >= qty]

    if not candidates:
        return None, None

    # Pick nearest warehouse from candidates
    df = distance[
        (distance["factory_id"] == preferred_factory) &
        (distance["warehouse_id"].isin(candidates))
    ]

    if df.empty:
        wh_choice = max(candidates, key=lambda w: capacity_map[w])
        return wh_choice, None

    best_row = df.sort_values("distance_km").iloc[0]
    return best_row["warehouse_id"], best_row["distance_km"]


In [6]:
recs = []

for _, row in incoming.iterrows():
    rec_wh, dist = recommend_warehouse(row)
    recs.append({
        "incoming_id": row["incoming_id"],
        "recommended_warehouse": rec_wh,
        "distance_km": dist
    })

recs_df = pd.DataFrame(recs)
recs_df.head()


,incoming_id,recommended_warehouse,distance_km
0,IN0001,WH10,6.07
1,IN0002,WH10,44.04
2,IN0003,WH08,13.81
3,IN0004,WH08,13.81
4,IN0005,WH08,13.81


In [7]:
# ---------- 6. MERGE BACK WITH INCOMING ----------
incoming_with_rec = incoming.merge(recs_df, on="incoming_id", how="left")

# Add free capacity of chosen warehouse
incoming_with_rec = incoming_with_rec.merge(
    warehouses_cap[["warehouse_id", "free_capacity_kg"]],
    left_on="recommended_warehouse",
    right_on="warehouse_id",
    how="left"
).rename(columns={"free_capacity_kg": "wh_free_capacity_kg"})

incoming_with_rec.drop(columns=["warehouse_id"], inplace=True)

# Alert if warehouse goes below 10% free capacity
incoming_with_rec["free_after_load_kg"] = (
    incoming_with_rec["wh_free_capacity_kg"] - incoming_with_rec["quantity_kg"]
)

incoming_with_rec["capacity_risk_flag"] = (
    incoming_with_rec["free_after_load_kg"] < 0.1 * incoming_with_rec["wh_free_capacity_kg"]
)

incoming_with_rec.head()


,incoming_id,timestamp,farmer,sku_id,sku_type,quantity_kg,preferred_factory_id,origin_lat,origin_lon,recommended_warehouse,distance_km,wh_free_capacity_kg,free_after_load_kg,capacity_risk_flag
0,IN0001,2025-11-02 09:53:52,Farmer_12,SKU05,spices,100,F03,22.674995,72.567548,WH10,6.07,4519,4419,False
1,IN0002,2025-10-30 10:53:52,Farmer_19,SKU04,spices,100,F02,23.275488,72.614708,WH10,44.04,4519,4419,False
2,IN0003,2025-11-03 13:53:52,Farmer_22,SKU02,oilseeds,300,F01,23.072585,71.997806,WH08,13.81,4699,4399,False
3,IN0004,2025-11-15 10:53:52,Farmer_32,SKU01,oilseeds,50,F01,22.457147,73.188133,WH08,13.81,4699,4649,False
4,IN0005,2025-11-01 13:53:52,Farmer_16,SKU02,oilseeds,100,F01,23.466611,72.648681,WH08,13.81,4699,4599,False


In [8]:
incoming_with_rec.to_csv("recommendations_full.csv", index=False)
warehouses_cap.to_csv("warehouse_capacity_status.csv", index=False)

print("Saved: recommendations_full.csv and warehouse_capacity_status.csv")


Saved: recommendations_full.csv and warehouse_capacity_status.csv
